Lo que hemos visto en la sección anterior, es una forma más manual de hacer la normalización.

A continuación, condensaremos el conocimiento anterior.


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.manual_seed(3)

transform = transforms.ToTensor()

train_full = datasets.CIFAR10("./data", train=True, download=True, transform=transform)


#Obtenemos el dataset train completo
loader_train = DataLoader(train_full, batch_size=len(train_full), shuffle=False)                       
#Obtenemos un batch de datos
imgs, _ = next(iter(loader_train))             
imgs = imgs.to(torch.float64)
print(imgs.shape) # [50000,3,32,32]

CIFAR10_MEAN = imgs.mean(dim=(0,2,3))
CIFAR10_STD  = imgs.std(dim=(0,2,3))
print(CIFAR10_MEAN, CIFAR10_STD)


cuda


In [27]:

#CIFAR10_MEAN = torch.tensor([0.4914, 0.4822, 0.4465])
#CIFAR10_STD  = torch.tensor([0.2470, 0.2435, 0.2616])

#Creamos el transform para normalización

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN.tolist(), CIFAR10_STD.tolist())
])

# Re-creamos los datasets con el nuevo transform
train_full = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)


#Comprobamos que los datos estan normalizados
check_loader_train_full = DataLoader(train_full, batch_size=len(train_full), shuffle=False) #Dataset completo para calcular la media y la desviación estándar de los datos ya normalizados
x, _ = next(iter(check_loader_train_full))
mean_check = x.mean(dim=(0, 2, 3))
std_check  = x.std(dim=(0, 2, 3))
print("Mean Appx: 0):", mean_check)
print("Std Appx: 1):", std_check)
assert mean_check.abs().max() < 0.05
assert (std_check - 1).abs().max() < 0.05



#Dividimos el dataset en train y validation
train_set, val_set = torch.utils.data.random_split(generator=torch.Generator().manual_seed(3), dataset=train_full, lengths=[40000, 10000])
#Comprobamos que el dataset se ha dividido correctamente
print(len(train_set), len(val_set))

#Nuestros loaders para entrenar, validar y testear, ya normalizados.
loader_train = DataLoader(train_set, batch_size=128, shuffle=True) #Barajamos porque es train y mejora la generalización
loader_val = DataLoader(val_set, batch_size=256, shuffle=False)
loader_test = DataLoader(test_set, batch_size=256, shuffle=False)








Mean Appx: 0): tensor([ 6.3542e-08, -1.9292e-08, -4.8446e-08])
Std Appx: 1): tensor([1.0000, 1.0000, 1.0000])
40000 10000


Vamos ahora a coger el primer batch

In [28]:
images, labels = next(iter(loader_train))
print(images.shape, labels.shape)

torch.Size([128, 3, 32, 32]) torch.Size([128])


# Declaración de arquitectura

In [32]:

import torch.nn as nn

def get_flatten_size(model_features, input_shape=(1, 3, 32, 32)):
    with torch.no_grad():
        x = torch.zeros(input_shape)
        out = model_features(x)
        return out.view(out.size(0), -1).size(1)

class CIFAR10CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.features = nn.Sequential(

            #Primera capa de convolucion, entra una imagen de 3 canales y sale una imagen de 32 features
            #El kernel es de 3x3 y el padding es 1 (same padding)
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(), #Aplicamos la funcion de activacion ReLU
            nn.MaxPool2d(kernel_size=2, stride=2), #Aplicamos pooling max para reducir la dimensionalidad de la imagen

            #Segunda capa de convolucion, entra una imagen de 16 features y sale una imagen de 32 features
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(), #Aplicamos la funcion de activacion ReLU
            nn.MaxPool2d(kernel_size=2, stride=2), #Aplicamos pooling max para reducir la dimensionalidad de la imagen

            #Tercera capa de convolucion, entra una imagen de 32 features y sale una imagen de 64 features
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(), #Aplicamos la funcion de activacion ReLU
            nn.MaxPool2d(kernel_size=2, stride=2), #Aplicamos pooling max para reducir la dimensionalidad de la imagen

             nn.Dropout(p=0.3), #Aplicamos dropout para evitar el overfitting
        )

        n_flat = get_flatten_size(self.features)

        

        #Añadimos una capa lineal para clasificar
        self.classifier = nn.Sequential(
            nn.Flatten(), #Aplanamos la imagen para que sea un vector
          
            
            nn.Linear(in_features=n_flat, out_features=500), #Una capa lineal con n_flat entradas y 500 salidas
            nn.ReLU(), #Aplicamos la funcion de activacion ReLU
            nn.Dropout(p=0.4), #Aplicamos dropout para evitar el overfitting
            
            nn.Linear(in_features=500, out_features=10), #Una capa lineal con 500 entradas y 10 salidas
            # No aplicamos softmax, ya que la funcion de perdida que usamos (CrossEntropyLoss) lo aplica por defecto
            # nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x





# Instancia
model = CIFAR10CNN()
print(model)

x = torch.randn(4, 3, 32, 32)
logits = model(x)
print(logits.shape)

from torchsummary import summary

summary(model.to(device), (1, 28, 28))

CIFAR10CNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.3, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1024, out_features=500, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=500, out_features=10, bias=True)
  )
)
torch.Size([4, 10])


NameError: name 'device' is not defined